In [2]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
!pip install wandb


In [3]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


True

In [4]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
#cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py')




root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/cascade_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head[0].num_classes = 10
cfg.model.roi_head.bbox_head[1].num_classes = 10
cfg.model.roi_head.bbox_head[2].num_classes = 10
print(cfg.model.roi_head.bbox_head[0])

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

{'type': 'Shared2FCBBoxHead', 'in_channels': 256, 'fc_out_channels': 1024, 'roi_feat_size': 7, 'num_classes': 10, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [0.1, 0.1, 0.2, 0.2]}, 'reg_class_agnostic': True, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': False, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'SmoothL1Loss', 'beta': 1.0, 'loss_weight': 1.0}}


In [5]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


In [6]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [7]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-01-11 11:01:43,215 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2024-01-11 11:01:43,217 - mmcv - INFO - load model from: torchvision://resnet50
2024-01-11 11:01:43,218 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2024-01-11 11:01:43,381 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2024-01-11 11:01:43,397 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-01-11 11:01:43,425 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2024-01-11 11:01:43,431 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [8]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-01-11 11:01:48,296 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-01-11 11:01:48,300 - mmdet - INFO - Start running, host: root@instance-5492, work_dir: /data/ephemeral/home/baseline/mmdetection/work_dirs/cascade_rcnn_r50_fpn_1x_trash
2024-01-11 11:01:48,301 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook   

2024-01-11 11:02:05,528 - mmdet - WARNING - No meta information found in the runner. 
2024-01-11 11:02:05,529 - mmdet - WARNING - To log evaluation or checkpoint metadata in MMDetWandbHook, `EvalHook` or `DistEvalHook` in mmdet is required, please check whether the validation is enabled.
2024-01-11 11:02:23,203 - mmdet - INFO - Epoch [1][50/1221]	lr: 1.978e-03, eta: 1:25:59, time: 0.353, data_time: 0.051, memory: 2574, loss_rpn_cls: 0.4083, loss_rpn_bbox: 0.0482, s0.loss_cls: 0.6918, s0.acc: 85.8066, s0.loss_bbox: 0.1139, s1.loss_cls: 0.2773, s1.acc: 91.3857, s1.loss_bbox: 0.0330, s2.loss_cls: 0.1351, s2.acc: 89.4902, s2.loss_bbox: 0.0049, loss: 1.7125, grad_norm: 8.5922
2024-01-11 11:02:37,337 - mmdet - INFO - Epoch [1][100/1221]	lr: 3.976e-03, eta: 1:17:07, time: 0.283, data_time: 0.007, memory: 2574, loss_rpn_cls: 0.1485, loss_rpn_bbox: 0.0439, s0.loss_cls: 0.3776, s0.acc: 92.6201, s0.loss_bbox: 0.1766, s1.loss_cls: 0.0996, s1.acc: 96.7783, s1.loss_bbox: 0.0531, s2.loss_cls: 0.0291,

learning_rate,▄▇█████████████████████████▂▂▂▂▂▂▂▂▂▂▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,▇█▂▂▃▁▁▁▂▂▁▂▂▂▂▃▃▂▂▃▃▂▃▃▂▃▂▁▁▂▂▂▂▂▂▂▃▁▂▁
train/loss,▆█▆▇▇▇▆▅▆▆▅▆▅▆▅▅▆▄▄▅▅▄▄▅▃▅▄▃▃▂▂▃▂▂▂▁▂▁▁▁
train/loss_rpn_bbox,██▅▇▆▅▅▄▅▅▄▅▄▅▄▄▄▃▃▄▄▄▃▄▃▅▃▂▂▂▂▃▂▂▂▂▁▁▂▁
train/loss_rpn_cls,▇█▅▅▅▄▄▄▄▄▃▃▃▄▃▃▃▃▂▃▃▃▃▃▂▃▂▂▂▁▂▂▁▁▁▁▁▁▁▁
train/s0.acc,▄▂▃▂▁▁▃▃▂▂▃▂▃▁▃▂▂▄▄▃▃▄▄▃▅▃▄▅▅▇▆▆▆▇▇▇▇█▇█
train/s0.loss_bbox,██▆▅▆▅▅▄▄▄▄▅▄▅▄▄▅▃▃▄▄▃▃▄▃▄▃▂▃▁▂▃▂▂▂▁▁▁▁▁
train/s0.loss_cls,▇█▆▇▇▆▅▅▅▆▅▅▅▆▅▅▅▄▄▅▅▄▄▄▃▄▄▃▃▂▂▂▂▂▁▁▂▁▁▁
train/s1.acc,█▄▃▂▁▁▃▃▁▁▂▂▂▁▃▂▂▃▃▃▂▄▃▃▅▃▄▅▅▆▆▆▆▇▇▇▇▇▇█
train/s1.loss_bbox,▂▇▆▇▇█▆▅▆▆▅▆▆▆▆▆▇▄▅▅▆▄▄▅▄▆▄▃▄▂▂▄▂▂▃▁▂▁▁▁
